In [ ]:
input_file = "/tmp/circles.csv"                         
config_file = "../config.json"

In [ ]:
import sys,os,ctypes
from pathlib import Path

notebook_dir = os.getcwd()
lib_path = os.path.join(notebook_dir,"../gng_py_lib/target/release")
sys.path.insert(0,os.path.abspath(lib_path))
print(lib_path)

import gng_py


import numpy as np
import pandas as pd
import time

In [ ]:
in_set = pd.read_csv(input_file)
in_set = in_set.to_numpy().flatten()

In [ ]:
ctx = gng_py.PyContext()
ctx = gng_py.PyContext()
ctx.load_config(config_file)
ctx.init_dataset_vec(in_set)
start = time.time()
ctx.fit()
end = time.time()
model_string = ctx.get_model_string()
print(end-start)

In [ ]:
import sys,os
import matplotlib.pyplot as plt
import json

In [ ]:
data = json.loads(model_string)

In [ ]:
points = None
edges = None
edge_positions = None
rows = []
for neuron in data["model"]["neurons"]:
    position = neuron["position"]
    id = neuron['id']
    x = position[0]
    y = position[1]
    rows.append({"id": id, "x": x, "y": y})

points = pd.DataFrame(rows)

# Extract edges
row_edge = []
for edge in data['model']['edges']:
    edge_from = edge['from']
    edge_to = edge['to']
    row_edge.append({"start": edge_from, "to": edge_to})

edges = pd.DataFrame(row_edge)

# Calculate edge positions
row = []
for _, row_edge in edges.iterrows():
    try:
        row_from = points.loc[points['id'] == row_edge['start']].iloc[0]
        x_from = float(row_from['x'])
        y_from = float(row_from['y'])
        row_to = points.loc[points['id'] == row_edge['to']].iloc[0]
        x_to = float(row_to['x'])
        y_to = float(row_to['y'])

        row.append({"x_from": x_from, "y_from": y_from, "x_to": x_to, "y_to": y_to})
    except (IndexError, KeyError):
        continue

edge_positions = pd.DataFrame(row)
import matplotlib.pyplot as plt
plot = plt.figure()
ax = plot.add_subplot(111)

ax.clear()

# Plot points
ax.scatter(points['x'], points['y'])

# Plot connections
for _, row in edges.iterrows():
    row_from = points.loc[points['id'] == row['start']].iloc[0]
    x_from = float(row_from['x'])
    y_from = float(row_from['y'])
    row_to = points.loc[points['id'] == row['to']].iloc[0]
    x_to = float(row_to['x'])
    y_to = float(row_to['y'])

    ax.plot([x_from, x_to], [y_from, y_to], 'k-', alpha=0.6)
